In [1]:
# Import all required packages
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,RandomizedSearchCV, GridSearchCV
from scipy.stats import randint,uniform
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, PrecisionRecallDisplay, average_precision_score,precision_recall_curve,auc
import seaborn as sn
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree  import DecisionTreeClassifier
import warnings

import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from joblib import Parallel
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures,StandardScaler,MinMaxScaler
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,train_test_split
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import pipeline,set_config
from sklearn.utils import class_weight
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, PrecisionRecallDisplay, precision_score, \
    average_precision_score, auc, precision_recall_curve, make_scorer,SCORERS
from skopt import BayesSearchCV
import datetime, re, sys, holidays
from Functions import *
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

set_config(display="diagram")
np.random.seed (0)
Parallel(n_jobs=8, max_nbytes=50000)

warnings.filterwarnings("ignore")

c:\users\rawwi\onedrive\school\nus\current sem\python\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
categorical_cols = ['Vessel Type']
numeric_cols = ['Waiting Time (Days)']
vessel_cols = ['Total Vessels (Number)', 'Capesize',
                'Capesize Laden', 'Capesize Unladen', 'Panamax', 'Panamax Laden',
                'Panamax Unladen', 'Handymax', 'Handymax Laden', 'Handymax Unladen',
                'Handysize', 'Handysize Laden', 'Handysize Unladen']
# date_cols = ['Berth or Port Entry', 'Date']
date_cols = ['Date']
dwt_cols = ['Total Vessels (DWT)','Capesize (DWT)',
            'Capesize Laden (DWT)', 'Capesize Unladen (DWT)', 'Panamax (DWT)',
            'Panamax Laden (DWT)', 'Panamax Unladen (DWT)', 'Handymax (DWT)',
            'Handymax Laden (DWT)', 'Handymax Unladen (DWT)', 'Handysize (DWT)',
            'Handysize Laden (DWT)', 'Handysize Unladen (DWT)']
weather_cols = ['tempt', 'prcp', 'wspd']

predict_col = 'Wait Levels'

In [3]:
# Get data
data = pd.read_csv('clean_data.csv', index_col=0, parse_dates=date_cols)
weather_df = pd.read_csv('sao_paulo_weather.csv', parse_dates=['date'])
weather_df = weather_df[['date','tavg','prcp','wspd']].rename(columns={'date': 'Date',
                   'tavg': 'tempt',
                   'prcp': 'prcp',
                   'wspd': 'wspd'}, inplace=False)
weather_df = weather_df.fillna(0)
data = data.merge(weather_df, how='inner',left_on=['Date'], right_on=['Date'])

# Drop cols with only 1 value
for col in data.columns:
    if len(data[col].unique()) == 1:
        data.drop(col,axis=1, inplace=True)

data.drop('Berth or Port Entry',axis=1, inplace=True)

#convert DWT cols from obj to int
for col in dwt_cols:
    data[col] = data[col].str.replace(",","").astype(int)
#convert vessel_cols from float to int
for col in vessel_cols:
    data[col] = data[col].astype(int)

#print cleaned data
data.head()

,IMO,Waiting Time (Days),Vessel Type,Date,Total Vessels (Number),Total Vessels (DWT),Capesize,Capesize Laden,Capesize Unladen,Panamax,...,Handymax (DWT),Handymax Laden (DWT),Handymax Unladen (DWT),Handysize (DWT),Handysize Laden (DWT),Handysize Unladen (DWT),Wait Levels,tempt,prcp,wspd
0,9056399,0.2,3,2015-01-02,16,878272,0,0,0,5,...,432704,121785,310919,52823,30060,22763,0,27.0,0.0,15.1
1,9471252,3.4,2,2015-01-02,16,878272,0,0,0,5,...,432704,121785,310919,52823,30060,22763,1,27.0,0.0,15.1
2,9233399,5.6,3,2015-01-02,16,878272,0,0,0,5,...,432704,121785,310919,52823,30060,22763,2,27.0,0.0,15.1
3,8309141,0.5,1,2015-01-03,17,908718,0,0,0,5,...,474533,107174,367359,70297,36036,34261,0,23.3,3.0,13.8
4,9625970,3.9,2,2015-01-03,17,908718,0,0,0,5,...,474533,107174,367359,70297,36036,34261,1,23.3,3.0,13.8


In [5]:
# split to train & Test
X_train, X_test, y_train, y_test = train_test_split(data.drop(predict_col,axis=1, inplace=False), data[predict_col], test_size=0.2, random_state=0)

Create Pipeline

In [6]:
date_linear_processor = pipeline.Pipeline([('ProcessDates',ProcessDates()),('ProcessHolidays',ProcessHolidays()),('DropDate',DropCol(cols=date_cols))])
numeric_linear_processor = pipeline.Pipeline([('Polynomial',PolynomialFeatures(degree=2,interaction_only =True,include_bias=True)),('Scaler',StandardScaler())])
cat_linear_processor = OneHotEncoder()
vessel_linear_processor = pipeline.Pipeline([('CreateDummyCol',CreateDummyCol()),('ProcessVessel',ProcessVessel(vessel_cols=vessel_cols,dwt_cols=dwt_cols))])
weather_linear_processor = pipeline.Pipeline([('CreateDummyCol',CreateDummyCol()),('ProcessWeather',ProcessWeather()),('ProcessWSpd',ProcessWSpd())])

data_preprocessor = ColumnTransformer([
    ('numeric_linear_processor',numeric_linear_processor, numeric_cols),
    ('cat_linear_processor',cat_linear_processor, categorical_cols),
    ('date_linear_processor',date_linear_processor, date_cols),
    ('vessel_linear_processor',vessel_linear_processor, vessel_cols+dwt_cols),
    ('weather_linear_processor',weather_linear_processor, weather_cols)
])
data_preprocessor


ColumnTransformer(transformers=[('numeric_linear_processor',
                                 Pipeline(steps=[('Polynomial',
                                                  PolynomialFeatures(interaction_only=True)),
                                                 ('Scaler', StandardScaler())]),
                                 ['Waiting Time (Days)']),
                                ('cat_linear_processor', OneHotEncoder(),
                                 ['Vessel Type']),
                                ('date_linear_processor',
                                 Pipeline(steps=[('ProcessDates',
                                                  ProcessDates()),
                                                 ('ProcessHolidays',
                                                  ProcessHolidays(...
                                  'Capesize Unladen (DWT)', 'Panamax (DWT)',
                                  'Panamax Laden (DWT)',
                                  'Panamax Unladen (DWT)', 'Handymax (DWT)',
                                  'Handymax Laden (DWT)',
                                  'Handymax Unladen (DWT)', 'Handysize (DWT)',
                                  'Handysize Laden (DWT)',
                                  'Handysize Unladen (DWT)']),
                                ('weather_linear_processor',
                                 Pipeline(steps=[('CreateDummyCol',
                                                  CreateDummyCol()),
                                                 ('ProcessWeather',
                                                  ProcessWeather()),
                                                 ('ProcessWSpd',
                                                  ProcessWSpd())]),
                                 ['tempt', 'prcp', 'wspd'])])

Regressor model

In [7]:
param_grid = {
    ## Preprocessing parameter options
    'columntransformer__date_linear_processor__ProcessDates': [ProcessDates(),None],
    'columntransformer__date_linear_processor__ProcessHolidays': [ProcessHolidays(),None],
    'columntransformer__weather_linear_processor__ProcessWeather__action': ['actual','drop'],
    'columntransformer__weather_linear_processor__ProcessWSpd__action': ['actual','cat','drop'],
    # 'columntransformer__vessel_linear_processor__ProcessWSpd__action': ['actual','cat','drop'],
    'columntransformer__vessel_linear_processor__ProcessVessel__usage': ['both','dwt','vessel'],


    ## ElasticNet hyper parameters
    'ElasticNet__regressor__alpha': np.linspace(0, 1, num=10),
    # 'ElasticNet__regressor__alpha': np.linspace(0, 1, num=2),
    'ElasticNet__regressor__l1_ratio': np.linspace(0, 1, num=10),

    ## XGBoost hyper parameters
    'XGBoost__regressor__reg_lambda': [0,0.1,0.5,1,5],
    'XGBoost__regressor__n_estimators': [200,400,600,800,1200],
    'XGBoost__regressor__learning_rate': np.linspace(0.05, 0.5, num=5),
    'XGBoost__regressor__max_depth': [-1,5,10,20,30,50,100],

    ## RandomForest hyper parameters
    'RandomForest__regressor__min_impurity_decrease': np.linspace(0.1, 0.5, num=4),
    'RandomForest__regressor__min_samples_leaf': [2,5,10,20,50],
    'RandomForest__regressor__min_samples_split': [2,5,10,20],
    'RandomForest__regressor__n_estimators': range(100,400,100),

    ## LGBM hyper parameters
    'LGBM__regressor__boosting_type': ['dart','goss','gbdt'],
    'LGBM__regressor__learning_rate': np.linspace(0.05, 0.5, num=5),
    'LGBM__regressor__max_depth': [-1,5,10,20,30,50,100],
    'LGBM__regressor____n_estimators': [200,400,600,800,1200],
}

In [8]:
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm  import LGBMRegressor
from sklearn.ensemble import StackingRegressor
estimators = [
    ("ElasticNet", ElasticNet(random_state=0)),
    ("XGBoost", XGBRegressor(objective='reg:squarederror',eval_metric='rmse',random_state=0,n_jobs=-1,use_label_encoder=False,  tree_method = 'gpu_hist')),
    ("RandomForest", RandomForestRegressor(random_state=0,n_jobs=-1)),
    ("LGBM", LGBMRegressor(objective='regression',random_state=0,n_jobs=-1,device = 'gpu')),
]
finalestimators = []
for estimator, model in estimators:
    finalestimators.append((estimator,pipeline.Pipeline([('columntransformer',data_preprocessor),('DropZeroCol',DropZeroCol()),('regressor',model)])))
final_estimator=LGBMRegressor(objective='regression',random_state=0,n_jobs=-1,device = 'gpu')
stacking_Regressor = StackingRegressor(estimators=finalestimators, final_estimator=final_estimator,n_jobs=-1)
# finalPipe = pipeline.make_pipeline(data_preprocessor,stacking_Regressor)
finalPipe = stacking_Regressor

StackingRegressor(estimators=[('ElasticNet',
                               Pipeline(steps=[('columntransformer',
                                                ColumnTransformer(transformers=[('numeric_linear_processor',
                                                                                 Pipeline(steps=[('Polynomial',
                                                                                                  PolynomialFeatures(interaction_only=True)),
                                                                                                 ('Scaler',
                                                                                                  StandardScaler())]),
                                                                                 ['Waiting '
                                                                                  'Time '
                                                                                  '(Days)']),
                                                                                ('cat_linear_processor',
                                                                                 OneHotEncoder(),
                                                                                 ['Vessel '
                                                                                  'Type']),
                                                                                ('date_linear_process...
                                                                                 Pipeline(steps=[('CreateDummyCol',
                                                                                                  CreateDummyCol()),
                                                                                                 ('ProcessWeather',
                                                                                                  ProcessWeather()),
                                                                                                 ('ProcessWSpd',
                                                                                                  ProcessWSpd())]),
                                                                                 ['tempt',
                                                                                  'prcp',
                                                                                  'wspd'])])),
                                               ('DropZeroCol', DropZeroCol()),
                                               ('regressor',
                                                LGBMRegressor(device='gpu',
                                                              objective='regression',
                                                              random_state=0))]))],
                  final_estimator=LGBMRegressor(device='gpu',
                                                objective='regression',
                                                random_state=0),
                  n_jobs=-1)

In [9]:
#tune hyperparameter
limitTuner = 'LGBM' # Options ElasticNet, XGBoost, RandomForest, LGBM
preprocessingName = 'columntransformer__'
estimatorsParam = {}
finalParam = {}
for estimator in finalestimators:
    if limitTuner== '' or estimator[0] ==limitTuner:
        modelParam = {}
        for param, grid in param_grid.items():
            if param.startswith(estimator[0]):
                modelParam[param[len(estimator[0])+2:]] = grid
            elif param.startswith(preprocessingName):
                modelParam[param] = grid
        # print(estimator[1].get_params().keys())
        print('Tuning Model ',estimator[0], "with params:\n",  modelParam, '\n')
        gs = GridSearchCV(estimator=estimator[1],param_grid=modelParam,scoring = 'neg_root_mean_squared_error',n_jobs = -1,cv = 5,verbose = -1)
        gs.fit(X_train,y_train)
        print('Tuned Model ',estimator[0], " (RMSE = ","{:.4f}".format(abs(gs.best_score_)),")")
        # print("Best params:\n",  gs.best_params_, '\n')
        modelParam={}
        for param, grid in gs.best_params_.items():
            modelParam[estimator[0]+'__'+param] = grid
        estimatorsParam ={**estimatorsParam, **modelParam}

print('\n Params to use\n\n',estimatorsParam)

Tuning Model  ElasticNet with params:
 {'columntransformer__date_linear_processor__ProcessDates': [ProcessDates(), None], 'columntransformer__date_linear_processor__ProcessHolidays': [ProcessHolidays(), None], 'columntransformer__weather_linear_processor__ProcessWeather__action': ['actual', 'drop'], 'columntransformer__weather_linear_processor__ProcessWSpd__action': ['actual', 'cat', 'drop'], 'columntransformer__vessel_linear_processor__ProcessVessel__usage': ['both', 'dwt', 'vessel'], 'regressor__alpha': array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
       0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ]), 'regressor__l1_ratio': array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
       0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ])} 



KeyboardInterrupt: 

Stop Here

In [74]:
for param, grid in gs.best_params_.items():
            modelParam[estimator[0]+'__'+param] = grid
estimatorsParam ={**estimatorsParam, **modelParam}
estimatorsParam

{'columntransformer__date_linear_processor__ProcessDates': None,
 'columntransformer__date_linear_processor__ProcessHolidays': None,
 'columntransformer__vessel_linear_processor__ProcessVessel__usage': 'dwt',
 'columntransformer__weather_linear_processor__ProcessWSpd__action': 'cat',
 'columntransformer__weather_linear_processor__ProcessWeather__action': 'actual',
 'ElasticNet__regressor__alpha': 0.0,
 'LGBM__columntransformer__date_linear_processor__ProcessDates': None,
 'LGBM__columntransformer__date_linear_processor__ProcessHolidays': None,
 'LGBM__columntransformer__vessel_linear_processor__ProcessVessel__usage': 'dwt',
 'LGBM__columntransformer__weather_linear_processor__ProcessWSpd__action': 'cat',
 'LGBM__columntransformer__weather_linear_processor__ProcessWeather__action': 'actual',
 'LGBM__regressor__alpha': 0.0}

In [29]:
# replace individual best model here post tuning.
elasticnetParams = {'ElasticNet__columntransformer__date_linear_processor__ProcessDates': None, 'ElasticNet__columntransformer__date_linear_processor__ProcessHolidays': None, 'ElasticNet__columntransformer__vessel_linear_processor__ProcessVessel__usage': 'dwt', 'ElasticNet__columntransformer__weather_linear_processor__ProcessWSpd__action': 'cat', 'ElasticNet__columntransformer__weather_linear_processor__ProcessWeather__action': 'actual', 'ElasticNet__regressor__alpha': 0.0, 'ElasticNet__regressor__l1_ratio': 0.0}
xgboostParams = {}
randomforestParams = {}
lgbmParams = {}


In [47]:
# prepare data for final tuning
estimatorsParam = {**elasticnetParams,**xgboostParams,**randomforestParams,**lgbmParams}
finalPipe.set_params(**estimatorsParam)
finalTrain  = finalPipe.fit_transform()
finalGrid = {}

# generate final model params
for param, grid in param_grid.items():
    if param.startswith("final_estimator"):
        finalGrid[param[len("final_estimator__"):]] = grid

#tune final model
gs = GridSearchCV(estimator=final_estimator,param_grid=finalGrid,scoring = 'rmse',n_jobs = -1,cv = 5,verbose = -1)
gs.fit(finalTrain)
modelParam = {}
for param, grid in gs.best_params_:
    modelParam["final_estimator__"+param] = grid
finalParam = {**estimatorsParam, **modelParam}

NameError: name 'finalTrain' is not defined